<a href="https://colab.research.google.com/github/AssafYeredor/AssafYeredor/blob/main/training_question_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_train.json to sample_train.json


In [ ]:
import json

def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

data = load_dataset('sample_train.json')

In [ ]:
import json
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

In [ ]:
def prepare_dataset(json_data):
    contexts, questions = [], []
    for item in json_data:
        for question in item['questions']:
            contexts.append(item['context'])
            questions.append(question)
    return Dataset.from_dict({'context': contexts, 'question': questions})

train_dataset = prepare_dataset(data)

In [ ]:
print(train_dataset[1])

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'What areas did Beyonce compete in when she was growing up?'}


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def preprocess_function(examples):
    # Concatenate the prefix with each context in the batch
    input_texts = ["generate question: " + context for context in examples['context']]
    model_inputs = tokenizer(input_texts, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

    # Tokenize questions in the batch
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['question'], max_length=128, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/4719 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',              # Directory where the model predictions and checkpoints will be written.
    evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_strategy="no",
    weight_decay=0.01,
    logging_dir=None,
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
10,12.533900
20,5.119000
30,1.525800
40,0.947100
50,0.642700
60,0.491700
70,0.468800
80,0.399700
90,0.448700
100,0.399500


TrainOutput(global_step=3540, training_loss=0.26515684144645085, metrics={'train_runtime': 2342.5966, 'train_samples_per_second': 6.043, 'train_steps_per_second': 1.511, 'total_flos': 8621016856657920.0, 'train_loss': 0.26515684144645085, 'epoch': 3.0})

In [ ]:
model = model.to('cuda')

def generate_question(model, tokenizer, context):
    input_text = "generate question: " + context
    # Ensure inputs are on the same device as the model
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=5, early_stopping=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question


# Example use
context_example = """Assaf is 20 years old."""
print(generate_question(model, tokenizer, context_example))


How old is Assaf?


In [ ]:
model.save_pretrained('./my_model_directory')
tokenizer.save_pretrained('./my_model_directory')


('./my_model_directory/tokenizer_config.json',
 './my_model_directory/special_tokens_map.json',
 './my_model_directory/spiece.model',
 './my_model_directory/added_tokens.json')

In [ ]:
from google.colab import files
import os

# Zip the model directory for easier download
!zip -r my_model_directory.zip my_model_directory

# Download the zip file
files.download('my_model_directory.zip')


  adding: my_model_directory/ (stored 0%)
  adding: my_model_directory/spiece.model (deflated 48%)
  adding: my_model_directory/model.safetensors (deflated 8%)
  adding: my_model_directory/added_tokens.json (deflated 83%)
  adding: my_model_directory/tokenizer_config.json (deflated 94%)
  adding: my_model_directory/special_tokens_map.json (deflated 85%)
  adding: my_model_directory/generation_config.json (deflated 30%)
  adding: my_model_directory/config.json (deflated 63%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def generate_questions(model, tokenizer, context, num_questions=5):
    input_text = "generate question: " + context
    # Ensure inputs are on the same device as the model
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(model.device)

    # Generate multiple outputs
    outputs = model.generate(
        inputs['input_ids'],
        max_length=128,
        num_beams=10,
        num_return_sequences=num_questions,
        temperature=1.5,  # Adjust temperature for more varied outputs
        early_stopping=True
    )

    # Decode each output to get the questions
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return questions

# Example usage
context_example = "Barak is visiting Lyon for a conference, which started on Monday and will end tomorrow."
questions = generate_questions(model, tokenizer, context_example, num_questions=5)
for question in questions:
    print(question)


What date is Barak in Lyon?
What is Barak doing in Lyon?
Where is Barak in Lyon?
What day is Barak in Lyon?
What day is Barak visiting Lyon?
